<a href="https://colab.research.google.com/github/HumzaW245/GraphNN-Project/blob/main/MAT6495_Final_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Processing

In [220]:
'''
Dataset for flights from: https://www.kaggle.com/datasets/muhammadbinimran/flight-price-prediction/code


'''
import kagglehub
data_path = kagglehub.dataset_download('muhammadbinimran/flight-price-prediction')

import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
%matplotlib inline


In [221]:
trainPath = None
testPath = None
for dirname, _, filenames in os.walk(data_path):
    for filename in filenames:
        path = os.path.join(dirname, filename)
        print(path)
        if "Train" in path:
          print(f'train patth is {path}' )
          trainPath = path
        else:
          print(f'test patth is {path}' )
          testPath = path


/root/.cache/kagglehub/datasets/muhammadbinimran/flight-price-prediction/versions/2/Data_Train.csv
train patth is /root/.cache/kagglehub/datasets/muhammadbinimran/flight-price-prediction/versions/2/Data_Train.csv
/root/.cache/kagglehub/datasets/muhammadbinimran/flight-price-prediction/versions/2/Test_set.csv
test patth is /root/.cache/kagglehub/datasets/muhammadbinimran/flight-price-prediction/versions/2/Test_set.csv


In [222]:
train=pd.read_csv(trainPath)

In [223]:
test=pd.read_csv(testPath)

In [224]:
flights =pd.concat([train,test])

In [225]:
flights.info()

<class 'pandas.core.frame.DataFrame'>
Index: 13354 entries, 0 to 2670
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Airline          13354 non-null  object 
 1   Date_of_Journey  13354 non-null  object 
 2   Source           13354 non-null  object 
 3   Destination      13354 non-null  object 
 4   Route            13353 non-null  object 
 5   Dep_Time         13354 non-null  object 
 6   Arrival_Time     13354 non-null  object 
 7   Duration         13354 non-null  object 
 8   Total_Stops      13353 non-null  object 
 9   Additional_Info  13354 non-null  object 
 10  Price            10683 non-null  float64
dtypes: float64(1), object(10)
memory usage: 1.2+ MB


In [226]:

# Reset the index
flights = flights.reset_index(drop=True)

flights = flights.drop_duplicates()

In [227]:

flights['Date_of_Journey'] = flights['Date_of_Journey'].apply(lambda dateStr: datetime.strptime(dateStr , '%d/%m/%Y'))


In [228]:
flights['Departure_Day']=flights['Date_of_Journey'].apply(lambda date: date.day)
flights['Departure_Month']=flights['Date_of_Journey'].apply(lambda date: date.month)
flights['Departure_Year']=flights['Date_of_Journey'].apply(lambda date: date.year)

In [229]:
def add_departure_time_to_date_of_journey(date_of_journey, dep_time):
  dep_time = datetime.strptime(dep_time, '%H:%M').time()
  date_flight_with_time = date_of_journey.replace(hour=dep_time.hour,minute=dep_time.minute)
  return date_flight_with_time


flights['Departure_Date'] = flights.apply(lambda row: add_departure_time_to_date_of_journey(row['Date_of_Journey'],row['Dep_Time']), axis=1)


In [230]:
def process_duration_string(duration):
  if 'm' in duration:
    duration = duration

  else:
    duration = duration + ' 0m'

  if 'h' in duration:
    duration = duration
  else:
    duration = '0h ' + duration
  duration = duration.replace('h ',':')
  duration = duration.replace('m','')
  dur_list = duration.split(':')
  hour = int(dur_list[0])
  min = int(dur_list[1])
  duration = timedelta(hours=hour, minutes=min)
  return duration

def get_arrival_time(date_flight, duration_string):
  duration = process_duration_string(duration_string)
  arrival_time = date_flight + duration
  return arrival_time

flights['Arrival_Date'] = flights.apply(lambda row: get_arrival_time(row['Departure_Date'], row['Duration']), axis = 1)


In [231]:
flights['Arrival_Day']=flights['Arrival_Date'].apply(lambda date: date.day)
flights['Arrival_Month']=flights['Arrival_Date'].apply(lambda date: date.month)
flights['Arrival_Year']=flights['Arrival_Date'].apply(lambda date: date.year)

In [232]:
def process_stops_string(stops):
  if stops == "1 stop":
    return 1
  elif stops == "2 stops":
    return 2
  elif stops == "3 stops":
    return 3
  elif stops == "4 stops":
    return 4
  elif stops == "non-stop":
    return 0
  elif pd.isna(stops):
    return 0
  else:
    print('unexpected stops value: ' + str(stops))
    assert False

flights['Total_Stops'] = flights['Total_Stops'].apply(lambda stops: process_stops_string(stops))


In [233]:
flights = flights[['Airline', 'Source', 'Destination',
       'Total_Stops',
       'Price', 'Departure_Date', 'Arrival_Date', 'Departure_Day', 'Departure_Month',
       'Departure_Year','Arrival_Day',
       'Arrival_Month', 'Arrival_Year']]

In [234]:
flights.head()

,Airline,Source,Destination,Total_Stops,Price,Departure_Date,Arrival_Date,Departure_Day,Departure_Month,Departure_Year,Arrival_Day,Arrival_Month,Arrival_Year
0,IndiGo,Banglore,New Delhi,0,3897.0,2019-03-24 22:20:00,2019-03-25 01:10:00,24,3,2019,25,3,2019
1,Air India,Kolkata,Banglore,2,7662.0,2019-05-01 05:50:00,2019-05-01 13:15:00,1,5,2019,1,5,2019
2,Jet Airways,Delhi,Cochin,2,13882.0,2019-06-09 09:25:00,2019-06-10 04:25:00,9,6,2019,10,6,2019
3,IndiGo,Kolkata,Banglore,1,6218.0,2019-05-12 18:05:00,2019-05-12 23:30:00,12,5,2019,12,5,2019
4,IndiGo,Banglore,New Delhi,1,13302.0,2019-03-01 16:50:00,2019-03-01 21:35:00,1,3,2019,1,3,2019


In [214]:
#flights.info()

<class 'pandas.core.frame.DataFrame'>
Index: 13108 entries, 0 to 13353
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Airline          13108 non-null  object        
 1   Source           13108 non-null  object        
 2   Destination      13108 non-null  object        
 3   Total_Stops      13108 non-null  int64         
 4   Price            10463 non-null  float64       
 5   Departure_Date   13108 non-null  datetime64[ns]
 6   Arrival_Date     13108 non-null  datetime64[ns]
 7   Departure_Day    13108 non-null  int64         
 8   Departure_Month  13108 non-null  int64         
 9   Departure_Year   13108 non-null  int64         
 10  Arrival_Day      13108 non-null  int64         
 11  Arrival_Month    13108 non-null  int64         
 12  Arrival_Year     13108 non-null  int64         
dtypes: datetime64[ns](2), float64(1), int64(7), object(3)
memory usage: 1.9+ MB


# Data Processing

In [ ]:
from sklearn.preprocessing import LabelEncoder
label=LabelEncoder()
final_df['Airline']=label.fit_transform(final_df['Airline'])
final_df['Source']=label.fit_transform(final_df['Source'])
final_df['Destination']=label.fit_transform(final_df['Destination'])
final_df['Additional_Info']=label.fit_transform(final_df['Additional_Info'])

In [ ]:
final_df

In [ ]:
final_df['Duration']=final_df['Duration_hour']*60+final_df['Duration_min']

In [ ]:
final_df.drop(['Duration_hour','Duration_min'],axis=1,inplace=True)

In [ ]:
final_df.info()

In [ ]:
final_df.isnull().sum()

In [ ]:
df1=pd.get_dummies(final_df,columns=["Airline","Source","Destination","Additional_Info"],drop_first=True,dtype=int)

In [ ]:
df1.info()

In [ ]:
df1.head()

In [ ]:
df1.isnull().sum()

In [ ]:
test=df1[df1['Price'].isnull()]

In [ ]:
test

In [ ]:
train=df1[~df1['Price'].isnull()]

In [ ]:
train

In [ ]:
X=train.drop(['Price'],axis=1)
y=train['Price']

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=42)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
model=RandomForestRegressor()
model.fit(X_train,y_train)
pred=model.predict(X_test)


In [ ]:
pred

In [ ]:
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
import numpy as np

# Calculate R^2 score
r2 = r2_score(y_test, pred)
print(f"R^2 score: {r2}")

# Calculate Mean Absolute Error
mae = mean_absolute_error(y_test, pred)
print(f"Mean Absolute Error: {mae}")

# Calculate Root Mean Squared Error
rmse = np.sqrt(mean_squared_error(y_test, pred))
print(f"Root Mean Squared Error: {rmse}")

In [ ]:
test=test.drop(['Price'],axis=1)

predictions=model.predict(test)
df=pd.DataFrame(predictions)
df.to_csv('output.csv')